In [52]:
#import libraries
import tensorflow as tf
from tensorflow.contrib import rnn
import pandas as pd
from google.colab import drive, files
import keras.models as km
import keras.layers as kl
import keras.optimizers as ko
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import keras.callbacks as kc
warnings.filterwarnings("ignore")
#get permission and load data
#drive.mount('/content/drive')
path = '/content/drive/My Drive/data/'
#training data and test data
xdata = np.load(path+'xdata_raw.npy')
ydata = np.load(path+'ydata_raw.npy')
#effective season length for each league
ngame = np.array([328,330,261,308,326])
#len8: leagues, len7: season, len9: match
len8,len7,len9,len_in = np.shape(xdata)
len8,len7,len9,len_out_full = np.shape(ydata)
X_train = np.zeros((len8,int(len7*len9*0.75),len_in))
X_test = np.zeros((len8,int(len7*len9*0.25),len_in))
#In trial1, we predict the observed odds for H/D/A
#not very well, using obs odds to predict who win isn't too good.
#In trial2, we predict the goal difference
#In trial 3, we predict who win
len_out = 3
Y_train = np.zeros((len8,int(len7*len9*0.75),len_out))
Y_test = np.zeros((len8,int(len7*len9*0.25),len_out))
Y_test_odds = np.zeros((len8,int(len7*len9*0.25),len_out))
count = np.zeros((2,len8)).astype(int)
#normalize and denormalize data
def normalize(value_in):
  val_max = np.max(value_in)
  val_min = np.min(value_in)
  len_game = len(value_in)
  ans = np.zeros((len_game))
  for i in range(len_game):
    if val_max!=val_min and np.isnan(value_in[i])==False and \
       np.isnan(val_max)==False and np.isnan(val_min)==False:
      ans[i]= 0.99*((value_in[i]-val_min)/(val_max-val_min))
      if np.isnan(ans[i])==True:
        ans[i]=0
  return ans,val_max,val_min
def denormalize(value_in, val_max, val_min):
  return val_min+value_in*(val_max-val_min)/0.99
#one-hot-encoding on who win
def find_winner(y,mode,goal_mean=None):
  if mode=='odds':
    len_game,len_cat = np.shape(y)
    ans = np.zeros((len_game,3))
    for i in range(len_game):
      if np.argmax(y[i,:])==0:
        ans[i,0] = 1
      elif np.argmax(y[i,:])==1:
        ans[i,1] = 1
      elif np.argmax(y[i,:])==2:
        ans[i,2] = 1
  if mode=='goal':
    len_game = len(y)
    ans = np.zeros((len_game,7))
    for i in range(len_game):
      if y[i]<=-3:
        ans[i,0] = 1
      elif y[i]>=3:
        ans[i,5] = 1
      else:
        ans[i,int(y[i]+3)] = 1
  if mode=='win_in':
    len_game = len(y)
    ans = np.zeros((len_game,3))
    for i in range(len_game):
      if y[i]==1:
        ans[i,0] = 1
      elif y[i]==0.5:
        ans[i,:] = 1
      elif y[i]==0:
        ans[i,2] = 1
  if mode=='win_out':
    len_game,len_var = np.shape(y)
    ans = np.zeros((len_game,3))
    for i in range(len_game):
      ans[i,int(y[i,:].argmax())]=1
  return ans
#retrieve X,Y train and test data
for i in range(len8):
  for j in range(len7):
    for k in range(len_in):
###############First option is to create data using train set split######################
      #xtrain_raw, xtest_raw, ytrain_raw, ytest_raw = train_test_split(xdata[i,j,:ngame[i],k], \
                                                                                 #ydata[i,j,:ngame[i],:], test_size=0.25)
      #X_train[i,count[0,i]:count[0,i]+int(0.75*ngame[i]),k],_,_ = normalize(xtrain_raw)
      #X_test[i,count[1,i]:count[1,i]+ngame[i]-int(0.75*ngame[i]),k],_,_ = normalize(xtest_raw)
    #Y_train[i,count[0,i]:count[0,i]+int(0.75*ngame[i]),:] =find_winner(ytrain_raw[:,4],mode='win_in')
    #Y_test[i,count[1,i]:count[1,i]+ngame[i]-int(0.75*ngame[i]),:] = find_winner(ytest_raw[:,4],mode='win_in')
    #Y_test_odds[i,count[1,i]:count[1,i]+ngame[i]-int(0.75*ngame[i]),:] = ytest_raw[:,1:4]
###############Second option #is to create train set data maunally###################
      xtrain_raw = xdata[i,j,:int(0.75*ngame[i]),k]
      xtest_raw = xdata[i,j,int(0.75*ngame[i]):int(0.75*ngame[i])+int(0.25*ngame[i]),k]
      ytrain_raw = ydata[i,j,:int(0.75*ngame[i]),:]
      ytest_raw = ydata[i,j,int(0.75*ngame[i]):int(0.75*ngame[i])+int(0.25*ngame[i]),:]
      X_train[i,count[0,i]:count[0,i]+int(0.75*ngame[i]),k],_,_ = normalize(xtrain_raw)
      X_test[i,count[1,i]:count[1,i]+int(0.25*ngame[i]),k],_,_ = normalize(xtest_raw)
    Y_train[i,count[0,i]:count[0,i]+int(0.75*ngame[i]),:] =find_winner(ytrain_raw[:,4],mode='win_in')
    Y_test[i,count[1,i]:count[1,i]+int(0.25*ngame[i]),:] = find_winner(ytest_raw[:,4],mode='win_in')
    Y_test_odds[i,count[1,i]:count[1,i]+int(0.25*ngame[i]),:] = ytest_raw[:,1:4]
    count[0,i] = count[0,i]+int(0.75*ngame[i])
    count[1,i] = count[1,i]+int(0.25*ngame[i])
            
###############NN hyperparameters##########################################
#number of iterations
hm_epochs=10
#output no. of variables
n_classes = len_out
#model simulation step (memory step)
batch_size = 50
#input no. of variables
chunk_size=len_in
#input moment size
n_chunks=1
#no. of neurons included in RNN
rnn_size=128
#drop out rate
do_rate = 0.2
###########################################################################
#Fully-connected layer, CNN or RNN
print ('Building network')
model = km.Sequential()
#model.add(kl.Conv1D(filters=64,kernel_size=2,activation='relu', input_shape=(n_chunks,chunk_size)))
#model.add(kl.MaxPooling1D(pool_size=1))
#model.add(kl.Flatten())
#model.add(kl.Embedding(len_in,output_dim=128))
#model.add(kl.LSTM(rnn_size))
model.add(kl.Dense(256))
model.add(kl.Dropout(do_rate))
model.add(kl.Dense(128))
model.add(kl.Dropout(do_rate))
model.add(kl.Dense(44))
model.add(kl.Dense(len_out,activation='softmax'))
#(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
optimizer = ko.Adam(lr=0.001)
#optimizer=ko.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#league title
title1=['EPL','La Liga','Bundesliga','Serie A','Ligue 1']
for i in range(5):
  early_stopping = kc.EarlyStopping(monitor='val_loss',patience=2)
  model.fit(X_train[i,:count[0,i],:], Y_train[i,:count[0,i],:], batch_size=batch_size, epochs=hm_epochs, callbacks=[early_stopping],verbose=0)
  model.evaluate(X_test[i,:count[1,i],:], Y_test[i,:count[1,i],:], batch_size=50)
  Y_test_mod = model.predict(X_test[i,:count[1,i],:])
  Y_test_mod_win = find_winner(Y_test_mod,mode='win_out')
  Y_test_odds_win = find_winner(Y_test_odds[i,:count[1,i],:],mode='win_out')
  print ('Accuracy for '+title1[i]+' using ML on game H/D/A', np.sum(np.equal(Y_test_mod_win,Y_test[i,:count[1,i],:]).astype(int))/(3*len(Y_test_mod_win[:,0])))
  print ('Accuracy for '+title1[i]+' using odds on game H/D/A', np.sum(np.equal(Y_test_odds_win,Y_test[i,:count[1,i],:]).astype(int))/(3*len(Y_test_mod_win[:,0])))
  
  

Building network
Epoch 1/10
1476/1476 [==============================] - 2s 2ms/step - loss: 1.5379 - acc: 0.6511
Epoch 2/10
1476/1476 [==============================] - 0s 63us/step - loss: 1.5013 - acc: 0.6944
Epoch 3/10
1476/1476 [==============================] - 0s 66us/step - loss: 1.4862 - acc: 0.6904
Epoch 4/10
1476/1476 [==============================] - 0s 65us/step - loss: 1.4796 - acc: 0.7046
Epoch 5/10
1476/1476 [==============================] - 0s 72us/step - loss: 1.4784 - acc: 0.6938
Epoch 6/10
1476/1476 [==============================] - 0s 64us/step - loss: 1.4741 - acc: 0.6965
Epoch 7/10
1476/1476 [==============================] - 0s 64us/step - loss: 1.4813 - acc: 0.6870
Epoch 8/10
1476/1476 [==============================] - 0s 67us/step - loss: 1.4717 - acc: 0.7087
Epoch 9/10
1476/1476 [==============================] - 0s 63us/step - loss: 1.4830 - acc: 0.6843
Epoch 10/10
492/492 [==============================] - 1s 2ms/step
Accuracy for EPL using ML on game H

(246, 3)
(246,)
